# AutoQuake Demo - Complete Earthquake Catalog Generation

Welcome to the AutoQuake demonstration! This notebook will guide you through the complete earthquake catalog generation workflow:

1. **Download sample seismic data**
2. **Phase Detection** with PhaseNet
3. **Event Association** with GaMMA  
4. **Location Refinement** with H3DD
5. **Magnitude Calculation**
6. **Polarity Analysis** with DiTingMotion
7. **Focal Mechanism** with GAfocal

## Prerequisites
- AutoQuake environment activated
- Internet connection for downloading demo data

Let's generate a complete earthquake catalog! 🌍📊

## Step 1: Download Demo Data and Setup

First, let's download sample seismic data and set up our working environment.

In [1]:
import os
import shutil
import sys
import tarfile
import urllib
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add AutoQuake to path
sys.path.append('.')

# Import AutoQuake components (following predict.py structure)
from autoquake.picker import PhaseNet
from autoquake.associator import GaMMA
from autoquake.focal import GAfocal
from autoquake.magnitude import Magnitude
from autoquake.polarity import DitingMotion
from autoquake.relocator import H3DD
from autoquake.utils import pol_mag_to_dout, process_for_h3dd_twice

print("🌍 AutoQuake Complete Demo")
print("=" * 50)
print("✅ All AutoQuake components imported successfully!")

🌍 AutoQuake Complete Demo
✅ All AutoQuake components imported successfully!


In [ ]:
# Demo Configuration - Set your parameters here!
DEMO_DATA_URL = "https://github.com/IES-ESLab/AutoQuake/releases/download/v1.0-demo/toy_data.tgz"
DEMO_DATA_DIR = Path("./toy_data")

# Download demo data if not exists
if not os.path.exists(DEMO_DATA_DIR):
    print("📥 Downloading demo data...")
    try:
        urllib.request.urlretrieve(DEMO_DATA_URL, "toy_data.tgz")
        with tarfile.open("toy_data.tgz", 'r:gz') as tar:
            tar.extractall()
        os.remove("toy_data.tgz")
        print(f"✅ Demo data extracted to '{DEMO_DATA_DIR}'")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please download manually from:", DEMO_DATA_URL)
else:
    print(f"✅ Demo data already exists in '{DEMO_DATA_DIR}'")

In [ ]:
NCPU = 2

RESULT_DIR = Path("demo_results")
RESULT_DIR.mkdir(exist_ok=True)

STATION = DEMO_DATA_DIR / "station.csv"
SAC_PARENT_DIR = DEMO_DATA_DIR / "data"
PZ_DIR = DEMO_DATA_DIR / "PZ"
H3DD_VEL_FILE = Path("./H3DD/tomops_H14")

## Step 2: Phase Detection with PhaseNet

PhaseNet is a deep learning model that automatically detects P and S wave arrivals in seismic data. This is the first step in earthquake catalog generation.

In [10]:
from ParamConfig.config_model import PhaseNetConfigReceiver
# Step 2: Run PhaseNet (Direct approach - no pydantic)
print("🚀 Running PhaseNet for phase detection...")

phasenet_config = PhaseNetConfigReceiver(
    data_parent_dir=SAC_PARENT_DIR,
    start="20240402",
    end="20240403",
    result_path=RESULT_DIR,
    pz_dir=PZ_DIR,    
)
# Initialize PhaseNet with direct parameters (like in predict.py)
phasenet = PhaseNet(
    phasenet_config
)

print("🔍 Detecting earthquake phases...")
print("This may take a few minutes depending on data size...")

try:
    # Run phase detection
    phasenet.predict()
    
    # Get results
    phase_picks = phasenet.get_picks()
    print(f"✅ PhaseNet completed!")
    print(f"📊 Detected {len(phase_picks)} phase picks")
    print(f"   P-waves: {len(phase_picks[phase_picks['phase_type'] == 'P'])}")
    print(f"   S-waves: {len(phase_picks[phase_picks['phase_type'] == 'S'])}")
    
except Exception as e:
    print(f"❌ PhaseNet error: {e}")
    print("Please check your data directory and configuration.")

🚀 Running PhaseNet for phase detection...
🔍 Detecting earthquake phases...
This may take a few minutes depending on data size...
Total samples:  None\.SAC : 0 files


Predicting: 0it [00:00, ?it/s]
Predicting: 0it [00:00, ?it/s]?it/s]
Merging 20240402: 0it [00:00, ?it/s]

Number of picks: 0
❌ PhaseNet error: 'PhaseNet' object has no attribute 'picks'
Please check your data directory and configuration.


## Step 3: Event Association with GaMMA

GaMMA (Gaussian Mixture Model Associator) groups individual phase picks into earthquake events using machine learning clustering techniques.

In [ ]:
def phasenet_station_name_processing(pickings: Path, output_dir: Path) -> Path:
    shutil.copy(pickings, output_dir/f"{pickings.stem}_original.csv")
    df = pd.read_csv(pickings)
    df['station_id'] = df['station_id'].map(lambda x: str(x).split('.')[1])
    output_path = output_dir/pickings.name
    df.to_csv(output_path, index=False)
    return output_path

# Step 3: Run GaMMA for event association
if 'phase_picks' in locals() and len(phase_picks) > 0:
    print("🔗 Running GaMMA for event association...")
    
    # Preprocessing for GaMMA (as in predict.py)
    post_phasenet_pickings = phasenet_station_name_processing(
        pickings=phase_picks,
        output_dir=RESULT_DIR
    )

    # Initialize GaMMA with direct parameters
    # choose number of processes: at most half of available CPUs, but not exceeding configured ncpu
    avail_cpus = os.cpu_count() or NCPU
    ncpu_to_use = min(max(1, avail_cpus // 2), NCPU)

    gamma = GaMMA(
        station=STATION,
        result_path=RESULT_DIR,
        center=(23.8, 121.0),
        xlim_degree=[120.0, 122.5],
        ylim_degree=[22.0, 25.5],
        pickings=post_phasenet_pickings,
        min_p_picks_per_eq=6,
        min_s_picks_per_eq=2,
        dbscan_eps=17.33,
        ncpu=ncpu_to_use,
        use_amplitude=False
    )
    
    try:
        # Run event association
        gamma.run_predict()
        
        # Get results
        gamma_events = gamma.get_events()
        gamma_picks = gamma.get_picks()
        
        print(f"✅ GaMMA completed!")
        print(f"🎯 Associated {len(gamma_events)} earthquake events")
        print(f"📍 Average picks per event: {len(gamma_picks) / len(gamma_events):.1f}")
        
    except Exception as e:
        print(f"❌ GaMMA error: {e}")
        
else:
    print("❌ No phase picks available. Please run PhaseNet first.")

## Step 4: Location Refinement with H3DD

H3DD (Hypocentroidal Decomposition Double Difference) refines earthquake locations using advanced algorithms and 3D velocity models.

In [ ]:
EVENT_NAME = "demo"

# Step 4: Run H3DD for location refinement (Two-step process as in predict.py)
if 'gamma_events' in locals() and len(gamma_events) > 0:
    print("Running H3DD location refinement...")
    
    try:
        # First H3DD run
        print("  ? H3DD first iteration...")
        h3dd = H3DD(
            gamma_event=gamma_events,
            gamma_picks=gamma_picks,
            result_path=RESULT_DIR,
            station=STATION,
            model_3d=H3DD_VEL_FILE,
            event_name=f"{EVENT_NAME}_c0",
            cut_off_distance_for_dd=0.0
        )
        h3dd.run_h3dd()
        
        # Process first results for second iteration
        print("  ⚙️  Processing first H3DD results...")
        h3dd_events_first, h3dd_picks_first = process_for_h3dd_twice(
            station=STATION,
            dout=h3dd.get_dout(),
            event_name_1=f"{EVENT_NAME}_c0",
            result_path=RESULT_DIR
        )
        
        # Second H3DD run for final refinement  
        print("  🔄 H3DD second iteration...")
        h3dd_2 = H3DD(
            gamma_event=h3dd_events_first,
            gamma_picks=h3dd_picks_first,
            result_path=RESULT_DIR,
            station=STATION,
            model_3d=H3DD_VEL_FILE,
            event_name=f"{EVENT_NAME}_c0_c2",
            cut_off_distance_for_dd=2.0
        )
        h3dd_2.run_h3dd()
        
        print(f"✅ H3DD completed! Refined locations for {len(h3dd_events_first)} events")
        
    except Exception as e:
        print(f"❌ H3DD error: {e}")
        print("Note: H3DD requires velocity models and may not work with minimal demo data")
        
else:
    print("❌ No events available. Please run GaMMA first.")

## Step 5: Magnitude & Polarity Analysis

Calculate earthquake magnitudes and analyze first-motion polarities for focal mechanism determination.

In [ ]:
def type_judge(station: str):
    """
    This is designed for determining station type based on its name.
    Typically, DAS stations start with a number, while traditional seismic stations start with a letter.
    """
    return station[0].isalpha()

# Step 5A: Magnitude Calculation
if 'h3dd_2' in locals():
    print("📏 Calculating earthquake magnitudes...")
    
    try:
        mag = Magnitude(
            dout_file=h3dd_2.get_dout(),
            station=STATION,
            sac_parent_dir=SAC_PARENT_DIR,
            pz_dir=PZ_DIR,
            output_dir=RESULT_DIR,
        )
        mag.run_mag(processes=2)  # Use 2 processes for demo
        
        mag_events = mag.get_events()
        mag_picks = mag.get_picks()
        print(f"✅ Magnitude calculation completed for {len(mag_events)} events")
        
    except Exception as e:
        print(f"⚠️  Magnitude calculation skipped: {e}")
        mag_events, mag_picks = None, None

# Step 5B: Polarity Analysis with DiTingMotion
if 'gamma_picks' in locals():
    print("🔍 Analyzing first-motion polarities...")
    
    try:
        dt_polarity = DitingMotion(
            gamma_picks=gamma_picks,
            output_dir=RESULT_DIR,
            sac_parent_dir=SAC_PARENT_DIR,
            type_judge=type_judge
        )
        dt_polarity.run_parallel_predict(processes=2)
        
        polarity_picks = dt_polarity.get_picks()
        print(f"✅ Polarity analysis completed for {len(polarity_picks)} picks")
        
    except Exception as e:
        print(f"⚠️  Polarity analysis skipped: {e}")
        polarity_picks = None
        
else:
    print("❌ No picks available for polarity analysis.")

## Step 6: Focal Mechanism with GAfocal

GAfocal uses genetic algorithms to determine earthquake focal mechanisms from polarity data.

In [ ]:
# Step 6: Focal Mechanism Determination
if all(var in locals() for var in ['h3dd_2', 'mag_events', 'polarity_picks']) and all(var is not None for var in [mag_events, polarity_picks]):
    print("🎯 Determining focal mechanisms...")
    
    try:
        # Format data for GAfocal (combines polarity and magnitude)
        print("  📝 Formatting data for focal mechanism inversion...")
        dout_file_name = pol_mag_to_dout(
            ori_dout=h3dd_2.get_dout(),
            result_path=RESULT_DIR,
            df_reorder_event=h3dd.get_df_reorder_event(),
            polarity_picks=polarity_picks,
            magnitude_events=mag_events,
            magnitude_picks=mag_picks
        )
        
        # Run GAfocal genetic algorithm
        print("  🧬 Running genetic algorithm for focal mechanisms...")
        gafocal = GAfocal(
            dout_file_name=dout_file_name,
            result_path=RESULT_DIR
        )
        gafocal.run()
        
        print("✅ Focal mechanism determination completed!")
        print(f"📊 Results saved to: {RESULT_DIR}")
        
    except Exception as e:
        print(f"⚠️  Focal mechanism calculation skipped: {e}")
        print("This step requires complete polarity and magnitude data.")
        
else:
    print("⚠️  Focal mechanism calculation skipped:")
    print("  Requires successful completion of all previous steps.")
    print("  For demo purposes, some steps may be skipped due to data limitations.")

## Step 7: Results Analysis & Summary

Let's examine the complete earthquake catalog we've generated!

In [ ]:
# Analyze Complete Results
print("📊 AutoQuake Complete Workflow Results")
print("=" * 50)

# Summary of all processing steps
results_summary = {
    'PhaseNet': len(phase_picks) if 'phase_picks' in locals() else 0,
    'GaMMA Events': len(gamma_events) if 'gamma_events' in locals() else 0,
    'H3DD Refined': len(h3dd_events_first) if 'h3dd_events_first' in locals() else 0,
    'Magnitudes': len(mag_events) if 'mag_events' in locals() and mag_events is not None else 0,
    'Polarities': len(polarity_picks) if 'polarity_picks' in locals() and polarity_picks is not None else 0
}

for step, count in results_summary.items():
    status = "✅" if count > 0 else "⚠️"
    print(f"{status} {step}: {count}")

# Check output files
print("\n📁 Generated Files:")
for file_path in RESULT_DIR.glob("*"):
    if file_path.is_file():
        size_mb = file_path.stat().st_size / (1024*1024)
        print(f"  📄 {file_path.name} ({size_mb:.2f} MB)")

# Create summary visualization if we have results
if 'phase_picks' in locals() and len(phase_picks) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('AutoQuake Complete Workflow Results', fontsize=16)
    
    # Plot 1: Workflow progress
    steps = list(results_summary.keys())
    counts = list(results_summary.values())
    axes[0, 0].bar(range(len(steps)), counts)
    axes[0, 0].set_title('Processing Pipeline Results')
    axes[0, 0].set_xticks(range(len(steps)))
    axes[0, 0].set_xticklabels(steps, rotation=45)
    axes[0, 0].set_ylabel('Count')
    
    # Plot 2: Phase picks timeline
    if len(phase_picks) > 0:
        phase_picks['phase_time_dt'] = pd.to_datetime(phase_picks['phase_time'])
        phase_picks.set_index('phase_time_dt').resample('H').size().plot(ax=axes[0, 1])
        axes[0, 1].set_title('Phase Detections Over Time')
        axes[0, 1].set_ylabel('Number of Picks')
    
    # Plot 3: Event locations (if available)
    if 'gamma_events' in locals() and len(gamma_events) > 0:
        axes[1, 0].scatter(gamma_events['longitude'], gamma_events['latitude'], 
                          c=gamma_events['depth_km'], cmap='viridis_r', s=50)
        axes[1, 0].set_title('Earthquake Locations')
        axes[1, 0].set_xlabel('Longitude')
        axes[1, 0].set_ylabel('Latitude')
        
    # Plot 4: Magnitude distribution (if available)
    if 'mag_events' in locals() and mag_events is not None and len(mag_events) > 0:
        axes[1, 1].hist(mag_events['magnitude'], bins=10, alpha=0.7)
        axes[1, 1].set_title('Magnitude Distribution')
        axes[1, 1].set_xlabel('Magnitude')
        axes[1, 1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

print("\n🎉 AutoQuake Demo Completed!")
print("\n📚 Next Steps:")
print("  • Modify parameters above to experiment with different settings")
print("  • Use your own seismic data following the directory structure")
print("  • Explore individual result files for detailed analysis")
print("  • Check the AutoQuake documentation for advanced features")
print("\n💡 Tips:")
print("  • Larger datasets will produce more reliable results")
print("  • Each step builds on the previous - ensure clean data flow")
print("  • Some steps may be skipped in demo due to data/model limitations")